# Sistema de Geração de Testes Unitários com Google Gemini

## 1. Instalar dependências

execute o comando abaixo para instalar as bibliotecas essenciais para a aplicação:


In [ ]:
# Instalar dependências necessárias
!pip install google-generativeai pytest

## 2. Configuração da API Google Gemini

Importe a biblioteca do Google Gemini e configure a chave de API usando os **Secrets do Google Colab**:


In [ ]:
import google.generativeai as genai
import os
import subprocess
from google.colab import userdata, files

# Configurar a API usando o Secrets do Colab
try:
    GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
    genai.configure(api_key=GEMINI_API_KEY)
    print("API key configurada com sucesso!")
except userdata.SecretNotFoundError:
    print("Erro: Chave GEMINI_API_KEY não encontrada nos Secrets.")
    print("Por favor, adicione a chave em: Menu > Ferramentas > Secrets")
except Exception as e:
    print(f"Erro ao configurar API: {e}")

API key configurada com sucesso!


## 3. Salvar e Executar Testes Unitários

Para salvar o código original e os testes gerados, e em seguida executar os testes usando **pytest**, utilizamos a função `save_and_run_tests`.  

Essa função realiza os seguintes passos:

1. Salva o código original no arquivo fornecido  
2. Salva os testes em um arquivo separado chamado `test_<nome_do_arquivo>`  
3. Executa os testes com `pytest -v`  
4. Retorna:
   - `stdout` → saída dos testes  
   - `stderr` → erros durante a execução  
   - `test_filename` → nome do arquivo de testes  
   - `success` → booleano indicando se todos os testes passaram  

> Em caso de tempo limite ou erro na execução, a função retorna informações de erro apropriadas.



In [ ]:
def save_and_run_tests(original_code, tests_code, filename):
    """
    Salva o código original e os testes, então executa os testes com pytest
    """
    try:
        # Salvar o código original
        with open(filename, "w") as f:
            f.write(original_code)

        # Salvar os testes
        test_filename = f"test_{filename}"
        with open(test_filename, "w") as f:
            f.write(tests_code)

        # Executar os testes
        result = subprocess.run(
            ["python", "-m", "pytest", test_filename, "-v"],
            capture_output=True,
            text=True,
            timeout=30
        )

        return result.stdout, result.stderr, test_filename, result.returncode == 0

    except subprocess.TimeoutExpired:
        return "", "Tempo limite excedido ao executar os testes", test_filename, False
    except Exception as e:
        return "", f"Erro ao executar testes: {str(e)}", test_filename, False



## 4. Gerar testes unitários automaticamente com Google Gemini

Esta etapa utiliza o modelo **Gemini 1.5 Flash** para criar testes unitários em **Python** para qualquer função ou módulo fornecido. Os testes seguem as boas práticas do `pytest` e são gerados automaticamente com base no comportamento real do código.

### Como funciona

1. Recebe o código-fonte Python e o nome do arquivo original.
2. Cria um prompt detalhado para o Gemini, instruindo o modelo a:
   - Analisar o comportamento real da função.
   - Gerar casos de teste apenas para funcionalidades implementadas.
   - Incluir testes de sucesso e falha (quando aplicável).
   - Usar `pytest.raises` somente para exceções realmente levantadas.
   - Garantir que o código gerado comece com `import pytest`.
3. Limpa o código gerado, removendo marcações de Markdown e explicações textuais.
4. Salva os testes em arquivo e permite execução automática para validação.

### Observações importantes

- Funções de teste sempre começam com `test_`.
- Casos de teste incluem asserts para verificar resultados esperados.
- É recomendado revisar os testes gerados antes de adicionar em produção, principalmente para códigos complexos.

Com esta abordagem, é possível acelerar a criação de testes unitários e garantir cobertura básica automática para funções Python.


In [ ]:
def generate_unit_tests(code: str, filename: str) -> str:
    """
    Gera testes unitários para um código Python usando o Gemini 1.5 Flash
    e remove quaisquer marcações de markdown da resposta
    """
    prompt = f"""
    Gere testes unitários completos usando pytest para o seguinte código Python.

    REGRAS IMPORTANTES:
    1. Analise cuidadosamente o comportamento real da função
    2. Teste apenas o que a função realmente faz, não assuma comportamentos
    3. Não crie testes para funcionalidades que não estão implementadas
    4. Para exceções, use pytest.raises apenas para exceções que a função realmente levanta
    5. Se a função não verifica tipos, não crie testes para tipos inválidos

    Inclua:
    1. Import pytest na primeira linha
    2. Importe as funções necessárias do código original
    3. Funções de teste com nomes que começam com 'test_'
    4. Casos de teste para sucesso e para falha quando aplicável
    5. Use asserts para verificar os resultados
    6. Trate casos de exceção com pytest.raises quando necessário

    O nome do arquivo original é: {filename}

    Retorne APENAS o código Python pronto para executar, sem explicações ou markdown.
    NÃO use ```python ou qualquer outra marcação de código.

    Código:
    {code}
    """

    try:
        # Usar o modelo Gemini 1.5 Flash
        model = genai.GenerativeModel('gemini-1.5-flash')
        response = model.generate_content(prompt)

        # Extrair o código dos testes
        tests_code = response.text.strip()

        # Remover marcações de markdown (```python e ```)
        tests_code = tests_code.replace("```python", "").replace("```", "")

        # Remover possíveis explicações textuais antes do código
        if "import pytest" in tests_code:
            tests_code = tests_code[tests_code.find("import pytest"):]

        # Garantir que começa com import pytest
        if not tests_code.startswith("import pytest"):
            tests_code = "import pytest\n" + tests_code

        return tests_code
    except Exception as e:
        return f"Erro ao gerar testes: {str(e)}"

# Função auxiliar para validar e limpar o código de testes
def clean_test_code(tests_code: str) -> str:
    """
    Remove marcações de markdown e garante que o código está limpo
    """
    # Remover marcações de markdown
    tests_code = tests_code.replace("```python", "").replace("```", "")

    # Remover linhas vazias no início
    tests_code = tests_code.strip()

    # Garantir que começa com import pytest
    if not tests_code.startswith("import pytest"):
        tests_code = "import pytest\n" + tests_code

    return tests_code

# Atualizar a função process_file para usar a limpeza
def process_file(file_path):
    """
    Processa um arquivo Python e gera testes automaticamente
    """
    try:
        # Ler o arquivo
        with open(file_path, "r") as f:
            code = f.read()

        # Extrair nome do arquivo
        filename = os.path.basename(file_path)

        print(f"Gerando testes para {filename} usando Gemini 1.5 Flash...")

        # Gerar testes
        tests = generate_unit_tests(code, filename)

        if tests.startswith("Erro ao gerar testes"):
            print(tests)
            return None, False

        # Limpar o código de testes
        clean_tests = clean_test_code(tests)

        # Salvar e executar testes
        output, error, test_filename, success = save_and_run_tests(code, clean_tests, filename)

        print(f"Testes salvos em {test_filename}")
        print("Saída dos testes:")
        print(output)

        if error:
            print("Erros:")
            print(error)

        if success:
            print("Todos os testes passaram!")
        else:
            print("Alguns testes falharam ou ocorreu um erro")

        return clean_tests, success

    except Exception as e:
        print(f"Erro ao processar arquivo: {e}")
        return None, False


## 5. Criar arquivos de exemplo para testes unitários

Antes de gerar os testes automáticos, criamos dois arquivos Python simples para servir de exemplo


In [ ]:
# Criar arquivo com função de soma
soma_code = """
def soma(a, b):
    return a + b
"""

with open("soma.py", "w") as f:
    f.write(soma_code)

# Criar arquivo com função de divisão
divisao_code = """
def divisao(a, b):
    if b == 0:
        raise ValueError("Divisão por zero não é permitida")
    return a / b
"""

with open("divisao.py", "w") as f:
    f.write(divisao_code)

## 6. Processar Arquivos de Exemplo

Para testar o sistema de geração automática de testes unitários, criamos dois arquivos de exemplo:

1. **soma.py** → contém a função `soma(a, b)`  
2. **divisao.py** → contém a função `divisao(a, b)` com tratamento de exceção para divisão por zero  

Em seguida, utilizamos a função `process_file` para gerar e executar os testes unitários para cada arquivo:


In [ ]:
# Processar os arquivos de exemplo
print("PROCESSANDO ARQUIVOS DE EXEMPLO")
process_file("soma.py")
process_file("divisao.py")


PROCESSANDO ARQUIVOS DE EXEMPLO
Gerando testes para soma.py usando Gemini 1.5 Flash...
Testes salvos em test_soma.py
Saída dos testes:
============================= test session starts ==============================
platform linux -- Python 3.12.11, pytest-8.4.2, pluggy-1.6.0 -- /usr/bin/python3
cachedir: .pytest_cache
rootdir: /content
plugins: typeguard-4.4.4, langsmith-0.4.27, anyio-4.10.0
collecting ... collected 5 items

test_soma.py::test_soma_inteiros_positivos PASSED                        [ 20%]
test_soma.py::test_soma_inteiros_negativos PASSED                        [ 40%]
test_soma.py::test_soma_inteiro_positivo_e_negativo PASSED               [ 60%]
test_soma.py::test_soma_zero PASSED                                      [ 80%]
test_soma.py::test_soma_com_zero PASSED                                  [100%]

============================== 5 passed in 0.11s ===============================

 Todos os testes passaram!
Gerando testes para divisao.py usando Gemini 1.5 Flash...
Te

('import pytest\nfrom divisao import divisao\n\ndef test_divisao_sucesso():\n    assert divisao(10, 2) == 5.0\n    assert divisao(5, 1) == 5.0\n    assert divisao(-10, 2) == -5.0\n    assert divisao(10, -2) == -5.0\n    assert divisao(0, 5) == 0.0\n\ndef test_divisao_zero():\n    with pytest.raises(ValueError) as excinfo:\n        divisao(10, 0)\n    assert str(excinfo.value) == "Divisão por zero não é permitida"',
 True)